In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import netCDF4 as nc
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import National_Snow_Model
import NSM_SCA
from datetime import datetime, timedelta
import os

In [ ]:
# Load the data
cwds = {"no_fSCA_Region_Pred": "/Users/jmac/Library/CloudStorage/Box-Box/Snow_Modeling_Group/REU_UA/ModelOutput/NSM2018_no_fSCA_Region_Pred",
        "fSCA_RegionVal": "/Users/jmac/Library/CloudStorage/Box-Box/Snow_Modeling_Group/REU_UA/ModelOutput/NSM2018_fSCA_RegionVal",
        "no_fSCA_RegionVal": "/Users/jmac/Library/CloudStorage/Box-Box/Snow_Modeling_Group/REU_UA/ModelOutput/NSM2018_no_fSCA_RegionVal"}

In [ ]:
dates = [datetime(2018, 12, 24), datetime(2019, 4, 15), datetime(2019, 7, 22)]

# Creates GeoJSON exports

In [ ]:
for cwd in cwds:
    for date in dates:
        # create SWE_Prediction class in the directory where the files are saved
        nsm = National_Snow_Model.SWE_Prediction(cwds[cwd], date.strftime('%Y-%m-%d'))

        nsm.Geo_df()  # create geodataframe from files

        # Export to GeoJSON
        nsm.SWE_gdf.to_file(cwds[cwd] + f'/NSM{date.strftime("%Y-%m-%d")}_{cwd}.geojson', driver='GeoJSON')

# os.system(f'say "your program has completely finished"')  # sometimes can take a while... only works on MacOS

# Combines Data from h5 files into a single DF

In [ ]:
from datetime import date, timedelta

#Grab existing files based on water year
prev_year = '2018'

#input the new water year of choice
water_year = '2019'

prev_date = date(int(prev_year), 10, 1)
new_date = date(int(water_year), 7, 31)

#write code for CSV files

#for h5 files
Region_list = ['N_Sierras',
                       'S_Sierras_High',
                       'S_Sierras_Low',
                       'Greater_Yellowstone',
                       'N_Co_Rockies',
                       'SW_Mont',
                       'SW_Co_Rockies',
                       'GBasin',
                       'N_Wasatch',
                       'N_Cascade',
                       'S_Wasatch',
                       'SW_Mtns',
                       'E_WA_N_Id_W_Mont',
                       'S_Wyoming',
                       'SE_Co_Rockies',
                       'Sawtooth',
                       'Ca_Coast',
                       'E_Or',
                       'N_Yellowstone',
                       'S_Cascade',
                       'Wa_Coast',
                       'Greater_Glacier',
                       'Or_Coast'
                      ]

In [ ]:
#can be altered to create list every n number of days by changing 7 to desired skip length
def daterange(start_date, end_date):
     for n in range(0, int((end_date - start_date).days) + 1, 7):
        yield start_date + timedelta(n)

#create empty list to store dates
datelist = []
#define start and end date for list of dates
start_dt = prev_date
end_dt = new_date
#append dates to list
for dt in daterange(start_dt, end_dt):
    #print(dt.strftime("%Y-%m-%d"))
    dt=dt.strftime('%Y-%m-%d')
    datelist.append(dt)

In [ ]:
datelist

In [ ]:
region_val = pd.read_pickle("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/Processed/RegionVal.pkl")

superset = []

for region in region_val:
    superset.append(region_val[region])

superset = pd.concat(superset)
superset = superset.set_index('cell_id')

In [ ]:
runs = {}

In [ ]:
for cwd in cwds:
    print(f"on {cwd}")
    run = superset.copy()
    for date in datelist:
        print(f"on {date}")
        run[date] = 0
        for region in Region_list:
            tmp = pd.read_hdf(f"{cwds[cwd]}/Predictions/predictions{date}.h5", key=region)
            tmp = tmp[[date]]
            run.update(tmp)
            # print(tmp.head())
    runs[cwd] = run

In [ ]:
megaset = pd.concat(runs)  # idk how to use multiindexing rn

In [ ]:
filled_megaset = megaset.fillna(0)

## Export to CSVs

In [ ]:
clean = {}
for run in runs:
    clean[run] = runs[run].drop(columns=['Lat','Long', 'elevation_m', 'northness'])
    runs[run].to_csv(f"/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/Processed/{run}.csv")

## Graph Volume

In [ ]:
totals = {}
for run in clean:
    total_in = clean[run].sum(axis=0)
    totals[run] = total_in

In [ ]:
df = pd.DataFrame(totals)

In [ ]:
df.rename(columns={'no_fSCA_Region_Pred': '5000ft Elevation Threshold', 'fSCA_RegionVal': 'fSCA Threshold', 'no_fSCA_RegionVal': 'Original Model'}, inplace=True)

In [ ]:
for col in df.columns:
    df[f"{col}_m"] = df[col] / 39.3700787402  # convert inches to meters
    df[f"{col} Volume"] = df[f"{col}_m"] / 1000  # convert meters to kilometers (using square kilometer grid, so we're done!)

In [ ]:
df.head()

In [ ]:
df.index = pd.to_datetime(df.index)
df.info()

In [ ]:
sns.set_style("whitegrid")
sns.set_context("poster")
sns.set(rc={'figure.figsize':(11.7,5.27)})

g = sns.lineplot(data=df[["Original Model Volume", "5000ft Elevation Threshold Volume", "fSCA Threshold Volume"]], dashes=False, )
t = plt.xticks(rotation=45, ha='right')
plt.ylabel("SWE (km^3)")
plt.xlabel("Date")
plt.title("Comparison of Model's SWE predictions for 2018-2019 Water Year")
plt.savefig("/Users/jmac/Documents/REU/exports/2018-2019_SWE_Predictions.png", dpi=600, bbox_inches='tight')
plt.show()